In [114]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [115]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [116]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [117]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:70090332243 -> bob:25467523847]

In [118]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:79118564073 -> alice:70090332243]

In [119]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
#bob._objects

In [120]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
#alice._objects

In [121]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x = x_ptr.get()
x

tensor([1, 2, 3, 4])

In [122]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [123]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [124]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [125]:
#bob._objects
#alice.objects

In [126]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:28770333745 -> bob:76781626575]

In [127]:
#print('  bob:', bob._objects)
#print('alice:',alice._objects)

In [128]:
x = x.move(alice)
x

(Wrapper)>[PointerTensor | me:28770333745 -> alice:76781626575]

In [129]:
#print('  bob:', bob._objects)
#print('alice:',alice._objects)

whenever we called a .send() or a .get() operation, it called that operation directly on the tensor on our local machine. However, if you have a chain of pointers, sometimes you want to call operations like .get() or .send() on the last pointer in the chain (such as sending data directly from one worker to another).